In [ ]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

client = OpenAI()

### Steps

#### 1. Define `Functions`
#### 2. Create an `Assitant` and register Functions
#### 3. Create a `Thread`
#### 4. Add a `Message` to the Thread
#### 5. `Run` the Assistant
#### 6. Polling for updates and `Calling` Functions